In [ ]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

     |████████████████████████████████| 106 kB 9.7 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
     |████████████████████████████████| 109 kB 39.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 1.5.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-translate 1.5.0 requires google-cloud-core<2.0dev,>=1.0.0, but you have google-cloud-core 2.2.1 which is incompatible.
google-cloud-language 1.2.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-firestore 1.7.0 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-firestore 1.7.0 requires google-cloud-core<2.0dev,>=1.0.3, but you have google-cloud-core 2.2.1 which

In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [ ]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2022-01-04 08:45:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  19.2MB/s    in 0.7s    

2022-01-04 08:45:55 (19.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 22KeqcMQYpd2iRnsaZZOzlyaNw0_5NAk69pXRUMvfrf1oCNpU

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

     |████████████████████████████████| 83 kB 916 kB/s 


# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c271-34-125-136-123.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Jan/2022 08:48:13] "GET /search?query=hello HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 08:48:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Jan/2022 08:49:10] "GET /search_body?query=hello HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 08:49:10] "GET /favicon.ico HTTP/1.1" 404 -


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [1]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [4]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [2]:
import requests
from time import time
url = 'http://35.192.205.59:8080'
# place the domain you got from ngrok or GCP IP below. 
# url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

# qs_res = []
# arr = list(queries.items())
# for i in range(len(arr)-15):
#   q, true_wids = arr[i][0],arr[i][1]
# # for q, true_wids in queries.items():
#   duration, ap = None, None
#   t_start = time()
#   try:
#     res = requests.get(url + '/search', {'query': q}, timeout=35)
#     duration = time() - t_start
#     if res.status_code == 200:
#       pred_wids, _ = zip(*res.json())
#       ap = average_precision(true_wids, pred_wids)
#   except:
#     pass
  
#   qs_res.append((q, duration, ap))


# print("search_body")
# qs_res = []
# arr = list(queries.items())
# for i in range(len(arr)-15):
#   q, true_wids = arr[i][0],arr[i][1]
#   print("query: ",q)
#   print("expected: ",true_wids)
# # for q, true_wids in queries.items():
#   duration, ap = None, None
#   t_start = time()
#   try:
#     res = requests.get(url + '/search_body', {'query': q}, timeout=35)
#     print(res)
#     duration = time() - t_start
#     if res.status_code == 200:
#       pred_wids, _ = zip(*res.json())
#       ap = average_precision(true_wids, pred_wids)
#       print("################### Ap: ", ap)
#       print("result pred_wids: ", pred_wids)
#   except:
#     pass
  
#   qs_res.append((q, duration, ap))
# print("out: ",true_wids)

# print("search_title")
# qs_res = []
# arr = list(queries.items())
# for i in range(len(arr)-15):
#   q, true_wids = arr[i][0],arr[i][1]
#   print("query: ",q)
#   print("expected: ",true_wids)
# # for q, true_wids in queries.items():
#   duration, ap = None, None
#   t_start = time()
#   try:
#     res = requests.get(url + '/search_title', {'query': q}, timeout=35)
#     print(res)
#     duration = time() - t_start
#     if res.status_code == 200:
#       pred_wids, _ = zip(*res.json())
#       ap = average_precision(true_wids, pred_wids)
#       print("################### Ap: ", ap)
#       print("result pred_wids: ", pred_wids)
#   except:
#     pass
  
#   # print("out result: ",res)

#   qs_res.append((q, duration, ap))



print("search_anchor")
qs_res = []
arr = list(queries.items())
for i in range(len(arr)-15):
  q, true_wids = arr[i][0],arr[i][1]
  print("query: ",q)
  print("expected: ",true_wids)
# for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_anchor', {'query': q}, timeout=35)
    duration = time() - t_start
    print(res)
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      print("################### Ap: ", ap)
      print("result pred_wids: ", pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))
# print("out: ",true_wids)

search_anchor
query:  best marvel movie
expected:  [60283633, 61073786, 5676692, 56289553, 60774345, 27306717, 61592102, 42163310, 60952488, 36439749, 48530084, 10589717, 29129051, 59892, 612052, 44254295, 878659, 54653881, 51430647, 66111204, 22114132, 55935213, 41677925, 17296107, 61651800, 9110929, 67055, 37497391, 60744481, 65464184, 41974496, 60616450, 60463979, 65967176, 57069491, 46208997, 22144990, 62372638, 1074657, 44240443, 33463661, 41974555, 43603241, 33700618, 5027882, 66423851, 60754840]
<Response [200]>
query:  How do kids come to world?
expected:  [15474, 1357127, 636806, 43033258, 6271835, 56480301, 23133297, 615418, 73165, 24470328, 1833777, 1380383, 79449, 4827661, 387703, 18863597, 36827305, 494299, 194687, 5591344, 48490547, 634139, 42072639, 44311171, 29384326, 1908019, 296627, 11263877, 101942, 2045465, 56921904, 128987, 22888933, 1072968, 25490788, 83449, 884998, 1151454, 30640885, 35072597, 2535885, 30861, 51046955, 13603, 3060346, 88380, 19698110, 72214, 6236

In [25]:
print("get_pagerank")
qs_res = []
arr = list(queries.items())

t_start = time()
try:
  res = requests.post('http://35.192.205.59:8080/get_pagerank', json(1000,4,5))
  pretty_json = json.loads(res.text)
  print (json.dumps(pretty_json, indent=2))
except:
  pass



# print(pred_wids)


get_pagerank


In [11]:
print("get_pageview")
qs_res = []
arr = list(queries.items())
# for i in range(len(arr)-15):
#   q, true_wids = arr[i][0],arr[i][1]
#   print("query: ",q)
#   print("expected: ",true_wids)
# for q, true_wids in queries.items():
duration, ap = None, None
t_start = time()
try:
  res = requests.post('http://35.192.205.59:8080/get_pageview', json(1,2,5))
  duration = time() - t_start
  print(res)
  # if res.status_code == 200:
  #   pred_wids, _ = zip(*res.json())
  #   ap = average_precision(true_wids, pred_wids)
  #   print("################### Ap: ", ap)
  #   print("result pred_wids: ", pred_wids)
except:
  pass



print(qs_res)

get_pageview
[]
